# Multi-agent systems.
The general idea is to create a system based on multiple agents that can handle specific tasks rather than using one single agent that can perform multiple ones.</br>

In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv

In [2]:
load_dotenv(Path.cwd().parent / ".env")

True

## Create tools

In [3]:
from typing import Annotated
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from langchain_experimental.utilities import PythonREPL

In [4]:
repl = PythonREPL()

@tool
def python_repl(code: Annotated[str, "Python code to execute"]):
    """
    This function will create a python script to generate a chart
    based on user query.
    """
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to excute code: {e}"
    
    result_str = f"Succesfully excecuted:\n```python\n{code}\n```Stdout: {result}"

    return (result_str + "\n\nIf you have completed all tasks, respond with FINAL ANSWER.")

## Graph State

In [5]:
import operator
from typing import Sequence, TypedDict
from langchain_core.messages import BaseMessage

In [6]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    sender: str

## Create Agents.
This system will have to agents:
1. The `researcher agent` access the web using the search tool to get information about user query.
2. The `chart agent` will use Python to cretae visuals.

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [10]:
llm = ChatOpenAI(model="gpt-4o-mini")

# -------------------------- Research agent -------------------------- #
tv_search = TavilySearchResults(max_results=5, search_depth="advanced", max_tokens=1e5)
search_tool = [tv_search]
search_tool_name = tv_search.name
research_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            f"""
            You are a helpful AI assistant, collaborating with other assistants.
            Use the provided tools to progress towards answering the question.
            If you are unable to fully answer, that's OK,
            another assistant with different tools will help where you left off.
            Execute what you can to make progress.
            If you or any of the other assistants have the final answer or deliverable,
            prefix your response with FINAL ANSWER so the team knows to stop.

            You have access to the following tools:
            {search_tool_name}

            You should provide accurate data for use
            and source code shouldn't be the final answer
            """
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

reseach_agent = (research_prompt | llm.bind_tools(search_tool))

# -------------------------- Chart agent -------------------------- #
chart_tool = [python_repl]
chart_tool_name = python_repl.name
chart_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            f"""
            You are a helpful AI assistant, collaborating with other assistants.
            Use the provided tools to progress towards answering the question.
            If you are unable to fully answer, that's OK,
            another assistant with different tools will help where you left off.
            Execute what you can to make progress.
            If you or any of the other assistants have the final answer or deliverable,
            prefix your response with FINAL ANSWER so the team knows to stop.

            You have access to the following tools:
            {chart_tool_name}

            Run the python code to display the chart
            """
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chart_agent = (chart_prompt | llm.bind_tools(chart_tool))